## Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.insert(0, '..')

In [36]:
import pickle
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from tqdm.notebook import tqdm
from src.features.schedules import (
    reason_pipeline_steps,
    numeric_pipeline_steps
)
from src.data_prep_utils import chunk_transform

In [22]:
df = pd.read_csv('../data/data_label_balanced.csv')
df.head()

,Unnamed: 0,partner_id,reason_combind,specialist_name
0,0,2,khám tiêu hóa,tiêu hoá
1,1,2,"đau lưng nhiều,ngồi lâu cứng lưng",cơ xương khớp
2,2,4,"hở van tim 3 lá,ngoại tâm thu",tim mạch
3,3,17,"đau tức ngực bên trái,cảm giác hồi hộp",tim mạch
4,4,17,cao huyết áp,tim mạch


In [25]:
df.isnull().sum()

Unnamed: 0          0
partner_id          0
reason_combind     10
specialist_name     0
dtype: int64

In [26]:
df = df.dropna(subset=['reason_combind'])

In [27]:
df.head()

,Unnamed: 0,partner_id,reason_combind,specialist_name
0,0,2,khám tiêu hóa,tiêu hoá
1,1,2,"đau lưng nhiều,ngồi lâu cứng lưng",cơ xương khớp
2,2,4,"hở van tim 3 lá,ngoại tâm thu",tim mạch
3,3,17,"đau tức ngực bên trái,cảm giác hồi hộp",tim mạch
4,4,17,cao huyết áp,tim mạch


## Persist

In [56]:
tfidf = TfidfVectorizer(
    min_df=5, max_features=1000, ngram_range=(1, 2)
)

In [57]:
tfidf.fit_transform(df['reason_combind'])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 642392 stored elements and shape (84419, 1000)>

In [49]:
tfidf

TfidfVectorizer(max_features=1000, min_df=5, ngram_range=(1, 2))

In [50]:
with open('../models/tfidf.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

In [51]:
with open('../models/tfidf.pkl', 'rb') as f:
    tfidf = pickle.load(f)

In [53]:
test = 'kham tieu hoa, lung cung'
res = tfidf.transform([test])

In [55]:
res.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [4]:
test_cases = pd.read_csv('../data/Testcases Newt ver 17 - Sheet1.csv', index_col=0)

In [5]:
test_cases = test_cases.iloc[:,:6]

In [6]:
test_cases.head()

,Câu hỏi,Lịch sử,Câu tóm tắt của AI,Output,Loại,Ghi chú
Thứ tự,,,,,,
1,Đau khớp gối kéo dài nên đi khám chuyên khoa nào?,[],NaN,Cơ xương khớp,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",NaN
2,Viêm khớp dạng thấp ảnh hưởng đến nhiều khớp c...,[],NaN,Cơ xương khớp,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",BV Chợ Rẫy PK Cơ xương khớp tự miễn
3,Đau đầu kéo dài và không rõ nguyên nhân nên đi...,[],NaN,Thần kinh,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",NaN
4,Rối loạn giấc ngủ nên đi khám chuyên khoa nào?,[],NaN,Thần kinh,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",đã khám thần kinh không đỡ chuyển khám sức khỏ...
5,"Đau bụng sau khi ăn, đầy hơi, khó tiêu cần đi ...",[],NaN,Tiêu hóa,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",NaN


In [7]:
test_cases['Output'] = test_cases['Output'].str.lower().str.strip()

In [8]:
test_cases.head()

,Câu hỏi,Lịch sử,Câu tóm tắt của AI,Output,Loại,Ghi chú
Thứ tự,,,,,,
1,Đau khớp gối kéo dài nên đi khám chuyên khoa nào?,[],NaN,cơ xương khớp,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",NaN
2,Viêm khớp dạng thấp ảnh hưởng đến nhiều khớp c...,[],NaN,cơ xương khớp,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",BV Chợ Rẫy PK Cơ xương khớp tự miễn
3,Đau đầu kéo dài và không rõ nguyên nhân nên đi...,[],NaN,thần kinh,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",NaN
4,Rối loạn giấc ngủ nên đi khám chuyên khoa nào?,[],NaN,thần kinh,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",đã khám thần kinh không đỡ chuyển khám sức khỏ...
5,"Đau bụng sau khi ăn, đầy hơi, khó tiêu cần đi ...",[],NaN,tiêu hóa,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",NaN


In [9]:
test_cases['partner_id'] = 0
test_cases['gender'] = -1
test_cases['age'] = -1
test_cases['province_id'] = -1

In [10]:
test_cases.head()

,Câu hỏi,Lịch sử,Câu tóm tắt của AI,Output,Loại,Ghi chú,partner_id,gender,age,province_id
Thứ tự,,,,,,,,,,
1,Đau khớp gối kéo dài nên đi khám chuyên khoa nào?,[],NaN,cơ xương khớp,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",NaN,0,-1,-1,-1
2,Viêm khớp dạng thấp ảnh hưởng đến nhiều khớp c...,[],NaN,cơ xương khớp,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",BV Chợ Rẫy PK Cơ xương khớp tự miễn,0,-1,-1,-1
3,Đau đầu kéo dài và không rõ nguyên nhân nên đi...,[],NaN,thần kinh,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",NaN,0,-1,-1,-1
4,Rối loạn giấc ngủ nên đi khám chuyên khoa nào?,[],NaN,thần kinh,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",đã khám thần kinh không đỡ chuyển khám sức khỏ...,0,-1,-1,-1
5,"Đau bụng sau khi ăn, đầy hơi, khó tiêu cần đi ...",[],NaN,tiêu hóa,"Bệnh/Triệu chứng đầy đủ, không gây nhiễu",NaN,0,-1,-1,-1


In [11]:
test_cases = test_cases.drop(columns=['Lịch sử', 'Câu tóm tắt của AI', 'Loại', 'Ghi chú'])

In [12]:
test_cases.head()

,Câu hỏi,Output,partner_id,gender,age,province_id
Thứ tự,,,,,,
1,Đau khớp gối kéo dài nên đi khám chuyên khoa nào?,cơ xương khớp,0,-1,-1,-1
2,Viêm khớp dạng thấp ảnh hưởng đến nhiều khớp c...,cơ xương khớp,0,-1,-1,-1
3,Đau đầu kéo dài và không rõ nguyên nhân nên đi...,thần kinh,0,-1,-1,-1
4,Rối loạn giấc ngủ nên đi khám chuyên khoa nào?,thần kinh,0,-1,-1,-1
5,"Đau bụng sau khi ăn, đầy hơi, khó tiêu cần đi ...",tiêu hóa,0,-1,-1,-1


In [13]:
test_cases.describe()

,partner_id,gender,age,province_id
count,220.0,220.0,220.0,220.0
mean,0.0,-1.0,-1.0,-1.0
std,0.0,0.0,0.0,0.0
min,0.0,-1.0,-1.0,-1.0
25%,0.0,-1.0,-1.0,-1.0
50%,0.0,-1.0,-1.0,-1.0
75%,0.0,-1.0,-1.0,-1.0
max,0.0,-1.0,-1.0,-1.0


In [14]:
test_cases = test_cases.rename(columns={"Câu hỏi": "reason_combind", "Output": "specialist_name"})

In [15]:
test_cases

,reason_combind,specialist_name,partner_id,gender,age,province_id
Thứ tự,,,,,,
1,Đau khớp gối kéo dài nên đi khám chuyên khoa nào?,cơ xương khớp,0,-1,-1,-1
2,Viêm khớp dạng thấp ảnh hưởng đến nhiều khớp c...,cơ xương khớp,0,-1,-1,-1
3,Đau đầu kéo dài và không rõ nguyên nhân nên đi...,thần kinh,0,-1,-1,-1
4,Rối loạn giấc ngủ nên đi khám chuyên khoa nào?,thần kinh,0,-1,-1,-1
5,"Đau bụng sau khi ăn, đầy hơi, khó tiêu cần đi ...",tiêu hóa,0,-1,-1,-1
...,...,...,...,...,...,...
216,"Tôi bị nghẹt mũi kéo dài, tôi nên khám chuyên ...",NaN,0,-1,-1,-1
217,Tôi hay bị khó thở nhẹ sau khi ăn. Tôi nên khá...,NaN,0,-1,-1,-1
218,"Khi trẻ có dấu hiệu chậm phát triển ngôn ngữ, ...",NaN,0,-1,-1,-1


## Set up model

In [71]:
SEED=42
params = {'n_estimators': 350, 'max_features': 'sqrt', 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 2}
model = RandomForestClassifier(
    random_state=SEED,
    **params
)

In [85]:
text_col = "reason_combind"
numeric_cols = ["partner_id", "gender", "age", "province_id"]

tfm = [
    (
        "reason_combind",
        Pipeline(reason_pipeline_steps()),
        text_col
    ),
    (
        "numeric_pipeline",
        Pipeline(numeric_pipeline_steps()),
        numeric_cols
    )
]

preprocessor = ColumnTransformer(
    transformers=tfm,
    remainder="drop"
)
features_pipeline = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("nomalize", StandardScaler()),
    ]
)

## training model

In [93]:
features = pd.read_csv('../data/data_label_balanced.csv', index_col=0)

from sklearn.model_selection import train_test_split

In [94]:
features = features.dropna(subset=["reason_combind"])

In [96]:
class_counts = features["specialist_id"].value_counts()
class_counts

specialist_id
1     4473
24    4473
4     4473
22    4472
18    4472
11    4472
3     4471
26    4470
27    4087
17    3078
31    2888
19    2539
29    2102
15    1630
32    1625
21    1288
5     1174
Name: count, dtype: int64

In [97]:
valid_classes = class_counts[class_counts >= 2].index
filtered_features = features[features['specialist_id'].isin(valid_classes)]

In [98]:
X = filtered_features.drop("specialist_id", axis=1)
y = filtered_features['specialist_id']

In [100]:
X

,partner_id,created_time,gender,birthtime,province_id,reason_combind,specialist_name
0,2,2016.0,-1,1978,1,khám tiêu hóa,tiêu hoá
2,2,2016.0,-1,1987,1,"đau lưng nhiều,ngồi lâu cứng lưng",cơ xương khớp
3,4,2016.0,-1,1987,40,"hở van tim 3 lá,ngoại tâm thu",tim mạch
4,17,2016.0,-1,1976,10,"đau tức ngực bên trái,cảm giác hồi hộp",tim mạch
6,17,2016.0,-1,1946,-1,cao huyết áp,tim mạch
...,...,...,...,...,...,...,...
198026,111,2023.0,1,-1,75,đi tiểu nhiều lần thận có sỏi,thận - tiết niệu
198028,372,2023.0,1,-1,79,chảy nước mắt sống,chuyên khoa mắt
198035,78,2023.0,1,-1,38,hay bị đau bụng,khám tổng quát
198036,348,2023.0,0,-1,79,vấn đề tâm thần,sức khỏe tâm thần


In [99]:
# fit the pipeline
fit_df = X.drop_duplicates(subset=["reason_combind"])
features_pipeline.fit(fit_df)

ValueError: A given column is not a column of the dataframe

## Testing

In [86]:
test_cases

,reason_combind,specialist_name,partner_id,gender,age,province_id
Thứ tự,,,,,,
1,Đau khớp gối kéo dài nên đi khám chuyên khoa nào?,cơ xương khớp,0,-1,-1,-1
2,Viêm khớp dạng thấp ảnh hưởng đến nhiều khớp c...,cơ xương khớp,0,-1,-1,-1
3,Đau đầu kéo dài và không rõ nguyên nhân nên đi...,thần kinh,0,-1,-1,-1
4,Rối loạn giấc ngủ nên đi khám chuyên khoa nào?,thần kinh,0,-1,-1,-1
5,"Đau bụng sau khi ăn, đầy hơi, khó tiêu cần đi ...",tiêu hóa,0,-1,-1,-1
...,...,...,...,...,...,...
216,"Tôi bị nghẹt mũi kéo dài, tôi nên khám chuyên ...",NaN,0,-1,-1,-1
217,Tôi hay bị khó thở nhẹ sau khi ăn. Tôi nên khá...,NaN,0,-1,-1,-1
218,"Khi trẻ có dấu hiệu chậm phát triển ngôn ngữ, ...",NaN,0,-1,-1,-1


In [87]:
X_test = test_cases.drop("specialist_name", axis=1)
y_test = test_cases['specialist_name']

In [88]:
X_test.shape, y_test.shape

((220, 5), (220,))

In [89]:
fit_df = X_test.drop_duplicates(subset=['reason_combind'])

fit_df

,reason_combind,partner_id,gender,age,province_id
Thứ tự,,,,,
1,Đau khớp gối kéo dài nên đi khám chuyên khoa nào?,0,-1,-1,-1
2,Viêm khớp dạng thấp ảnh hưởng đến nhiều khớp c...,0,-1,-1,-1
3,Đau đầu kéo dài và không rõ nguyên nhân nên đi...,0,-1,-1,-1
4,Rối loạn giấc ngủ nên đi khám chuyên khoa nào?,0,-1,-1,-1
5,"Đau bụng sau khi ăn, đầy hơi, khó tiêu cần đi ...",0,-1,-1,-1
...,...,...,...,...,...
216,"Tôi bị nghẹt mũi kéo dài, tôi nên khám chuyên ...",0,-1,-1,-1
217,Tôi hay bị khó thở nhẹ sau khi ăn. Tôi nên khá...,0,-1,-1,-1
218,"Khi trẻ có dấu hiệu chậm phát triển ngôn ngữ, ...",0,-1,-1,-1


In [90]:
features_pipeline.fit(fit_df)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('reason_combind',
                                                  Pipeline(steps=[('tfidf',
                                                                   TfidfVectorizer(max_features=128,
                                                                                   ngram_range=(1,
                                                                                                2),
                                                                                   strip_accents='unicode',
                                                                                   sublinear_tf=True)),
                                                                  ('todense',
                                                                   FunctionTransformer(func=<function todense at 0x7a5888121090>))]),
                                                  'reason_combind'),
                                                 ('numeric_pipeline',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['partner_id', 'gender',
                                                   'age', 'province_id'])])),
                ('nomalize', StandardScaler())])

In [91]:
X_test = chunk_transform(
    X_test, features_pipeline, chunk_size=1000
)

Transforming chunks:   0%|          | 0/1 [00:00<?, ?it/s]

In [92]:
y_pred = model.predict(X_test)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.